In [75]:
import pandas as pd
import numpy as np

# 정규식 활용 -> 특수문자 제거
import re

# 한글 형태소 분석기
from konlpy.tag import Okt

# 단어 빈도수 세기
from collections import Counter

In [79]:
date_str = '20250808'

In [80]:
# 파일 불러오기
raw_data_gen = pd.read_csv(f"/Users/leesangwon/Documents/ThemeStock_file/hankyung_general_{date_str}.csv")
raw_data_i = pd.read_csv(f"/Users/leesangwon/Documents/ThemeStock_file/hankyung_i_{date_str}.csv")
raw_data_g = pd.read_csv(f"/Users/leesangwon/Documents/ThemeStock_file/hankyung_g_{date_str}.csv")

# 하나의 파일로 유니온
raw_data = pd.concat([raw_data_gen, raw_data_i, raw_data_g], ignore_index=True)
print(raw_data.shape)

# 완전일치 중복기사 제거
df = raw_data.drop_duplicates(subset=['title', 'text'], keep='first').copy()

# title에 [속보] 또는 [포토] 포함 시 text를 NaN으로 변경
df.loc[df['title'].str.contains(r'\[속보\]|\[포토\]', regex=True), 'text'] = np.nan

# null셀을 빈칸으로 만들기
df['title'] = df['title'].fillna('')
df['text'] = df['text'].fillna('')

# 특수문자 등 전처리
df['text_clean'] = df['text'].apply(lambda t: re.sub(r'[^가-힣A-Za-z&\$₩]', ' ', t))
df['title_clean'] = df['title'].apply(lambda t: re.sub(r'[^가-힣A-Za-z&\$₩]', ' ', t))
df.head(3)

# 뉴스기사들 칼럼(text) 부분 전체 통합
text_all = " ".join(df['text_clean'].dropna().tolist())


###########################

# 형태소 분석기 생성
okt = Okt()

# 토큰화 - 명사만
tokens = okt.morphs(text_all)
tokens

# 불용어 불러오기
with open("/Users/leesangwon/Documents/ThemeStock_file/stopwords_kor.txt", "r", encoding="utf-8") as f:
    stopwords = f.read()

# 불용어 제거
tokens = [word for word in tokens if not word in stopwords]
tokens = " ".join(tokens)

# 다시 토큰화
cleaned_tokens = okt.morphs(tokens)
print(f'{len(cleaned_tokens)} 개 단어')


###########################

# 1. 빈도 계산
word_counts = Counter(cleaned_tokens)

# 2. DataFrame 변환
df_counts = pd.DataFrame(word_counts.items(), columns=['word', 'count'])

# 3. count 기준 내림차순 정렬
df_counts = df_counts.sort_values(by='count', ascending=False).reset_index(drop=True)

df_counts

(451, 4)
59010 개 단어


,word,count
0,미국,290
1,대통령,242
2,한국,215
3,AI,208
4,관세,156
...,...,...
13327,티니,1
13328,캐치,1
13329,성탄절,1
13330,틴업,1


In [81]:
df_counts.to_csv(f"/Users/leesangwon/Documents/ThemeStock_file/TF_{date_str}.csv", index=False)